In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
dataframe_location_2019_2020 = pd.read_csv("../../resources/original_data/FinlandNestDatafile.csv")
dataframe_location_2021 = pd.read_csv("../../resources/original_data/Finland_nestdata2021_mod.csv")
dataframe_mobbing_2019_2020 = pd.read_csv("../../resources/original_data/FinlandMobbingDatafile.csv")
dataframe_mobbing_2021 = pd.read_csv("../../resources/original_data/Finland_ExperimentData2021_mod.csv")

# combine mobbing dataframes:
dataframe_mobbing = pd.concat([dataframe_mobbing_2021,dataframe_mobbing_2019_2020], axis=0, ignore_index=True)
# combine locations:
dataframe_location_2021['Year'] = np.repeat(2021, dataframe_location_2021.shape[0])
dataframe_location = pd.concat([dataframe_location_2019_2020,dataframe_location_2021], axis=0, ignore_index=True)

dataframe_mobbing = dataframe_mobbing.drop(
    columns=['Site', 'Year', 'lat', 'long', 'Cuckoo_perch', 'New_rebuild', 'Rebuild_original']
)

data = pd.merge(dataframe_location, dataframe_mobbing, left_on='NestID', right_on='NestID', how = 'left')

In [ ]:
for i in range(data.shape[0]):
    if data['Propensity'][i] == 0:
        data.loc[i, 'mobbing'] = 'shy'
    elif data['Propensity'][i] == 1:
        data.loc[i, 'mobbing'] = 'aggressive'
    else:
        data.loc[i, 'mobbing'] = 'no info'

In [ ]:
data

In [ ]:
data[data.Year == 2021].Site.str.strip().value_counts()

In [ ]:
data[data.Year == 2020].Site.str.strip().value_counts()

In [ ]:
data[data.Year == 2019].Site.str.strip().value_counts()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_one_year(data, year, site, plot = False):
    tmp = data[(data.Year == year) & (data.Site == site)]
    
    if plot:
        fig = px.scatter_mapbox(tmp, lat="lat", lon="long", hover_name="NestID", hover_data=["Site", "Propensity"], 
                            height = 300, color = "mobbing", 
                            color_discrete_map={ 'shy': 'blue', 'aggressive': 'red', 'no info': 'grey'},
                            width = 500, zoom = 13, title = site + ', Year: ' + str(year))
        fig.update_layout(mapbox_style="open-street-map")
        fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0})
        fig.show()
    
    n_no_test = np.sum(pd.isnull(tmp.Propensity))
    n_shy = np.sum(tmp.Propensity == 0)
    n_aggressive = np.sum(tmp.Propensity == 1)
    
    return [n_no_test, n_shy, n_aggressive]

def plot_all_years(data, site):
    tmp = data[(data.Site == site)]
    
    fig = px.scatter_mapbox(tmp, lat="lat", lon="long", hover_name="NestID", hover_data=["Site", "Propensity"], 
                        height = 300, color = tmp.Year.astype(str), 
                        width = 500, zoom = 13, title = site)
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0})
    fig.show()

In [ ]:
from matplotlib.pyplot import figure
import plotly.express as px

def create_stacked_bar_chart(data, year):
    unique_sites = data[data.Year == year].Site.str.strip().unique()
    a = []
    
    for i in range(len(unique_sites)):
        #print([unique_sites[i]] + plot_one_year(data, year, unique_sites[i]))
        a.append([unique_sites[i]] + plot_one_year(data, year, unique_sites[i]))
    
    a = pd.DataFrame(a, columns=['Site', 'No_test', 'N_shy', 'N_aggressive'])

    
    a.plot(
        x = 'Site',
        kind = 'barh',
        stacked = True,
        title = 'Stacked Bar Graph',
        mark_right = True)

In [ ]:
create_stacked_bar_chart(data, 2019)

In [ ]:
create_stacked_bar_chart(data, 2020)

In [ ]:
create_stacked_bar_chart(data, 2021)

In [ ]:
plot_all_years(data, 'Myrans')

In [ ]:
site = 'Matasaarentie'

plot_one_year(data, 2019, site, True)
plot_one_year(data, 2020, site, True)
plot_one_year(data, 2021, site, True)

plot_all_years(data, site)

In [ ]:


fig = px.scatter_mapbox(data[data.Year == 2021], lat="lat", lon="long", hover_name="NestID", hover_data=["Site"], color="Site", zoom=3, height=600)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()